  # ДЗ №4. Траснформеры

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import torch
import time
import numpy as np
import pandas as pd
import random

from tqdm import tqdm
from transformers import AdamW, BertTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch import nn
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader
from math import log2, ceil
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

random.seed(123)
np.random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed_all(123)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")

    print(f'Доступно {torch.cuda.device_count()} GPU(s).')
    print('Используемый GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")

## EDA

Загрузим данные и разделим их на тренировочный, тестовый и валидационный наборы.

In [17]:
_data_train_val = pd.read_csv('./data/in_domain_train.csv')
_data_train_val = _data_train_val[:(int(len(_data_train_val) / 10))] #TODO

_X_train, _X_val, _y_train, _y_val = train_test_split(_data_train_val['sentence'], _data_train_val['acceptable'], test_size=0.1, random_state=123)

_X_train = _X_train.to_numpy()
_X_val = _X_val.to_numpy()
_y_train = _y_train.to_numpy()
_y_val = _y_val.to_numpy()

_data_test = pd.read_csv('./data/in_domain_dev.csv')
_X_test = _data_test['sentence']
_y_test = _data_test['acceptable']

In [ ]:
#print(len(_X_train))

707


Посмотрим на баланс классов.

In [30]:
#_y_train.hist()

Классы несбалансированы. Для оценки качества будем использовать MCC (Matthews Correlation Coefficient).

## BERT

### Обучение

Определим несколько утлитных классов.

In [20]:
class RuBertTokenizer():
    '''
    Принимает корпус текстов, токенизирует его и возвращает input_ids и attemtion_masks в виде тензоров.
    '''
    def __init__(self):
        # https://huggingface.co/ai-forever/ruBert-base
        self._bert_tokenizer = BertTokenizer.from_pretrained('ai-forever/ruBert-base')

    def transform(self, X):
        #max_length = self._get_max_document_length(X, self._bert_tokenizer)

        input_ids = []
        attention_masks = []

        for document in X:
            encoded_dict = self._bert_tokenizer.encode_plus(document,
                                                            add_special_tokens=True,
                                                            max_length=64, #TODO
                                                            pad_to_max_length=True,
                                                            return_tensors='pt',
                                                            truncation=True)
            input_ids.append(encoded_dict['input_ids'])
            attention_masks.append(encoded_dict['attention_mask'])

        # преобразуем в тензоры
        input_ids = torch.cat(input_ids, dim=0)
        attention_masks = torch.cat(attention_masks, dim=0)

        return input_ids, attention_masks
    
    def _get_max_document_length(self, X, tokenizer:BertTokenizer):
        # находим максимальную длину документа
        max_length = 0
        for document in X:
            tokenized_document = tokenizer.encode(document, add_special_tokens=True)
            max_length = max(max_length, len(tokenized_document))

        # Увеличиваем длину до ближайшей степени двойки.
        # Например, если максимальная длина документа 41, то берем 64.
        result = pow(2, ceil(log2(max_length)))

        if max_length != result:
            print(f'Максимальная длинна документа [{max_length}] уже является степенью двойки.')
        else:
            print(f'Максимальная длинна документа [{max_length}]. Берем [{result}].')

        return result

In [ ]:
class Trainer:
    '''
    Утилитный класс для обучения и валидации модели.
    '''

    def __init__(self, model, optimizer, num_of_epochs:int):
        self._model = model
        self._optimizer = optimizer
        self._num_of_epochs = num_of_epochs

    def train(self, test_data_loader:DataLoader, val_data_loader:DataLoader):
        self._model.to(device)
        self._model.train() # переводим модель в режим обучения

        for epoch in range(self._num_of_epochs):
            print(f'=== Эпоха {epoch} ===')
            print('Обучение...')

            self._train_epoch(epoch, test_data_loader)

            print('Валидация...')
            self.test(val_data_loader)

    def _train_epoch(self, epoch:int, data_loader:DataLoader):
        start_time = time.time()

        for batch in tqdm(data_loader):
            # обнуляем предыдущие значения градиентов
            self._model.zero_grad()

            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            # делаем предсказание
            # сравниваем предсказанные значения с истинными
            pred = self._model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = pred.loss

            total_loss += loss.item()

            # вычисляем градиент функции потерь
            loss.backward()
                
            # обновляем веса модели
            self._optimizer.step()

        elapsed_time_sec = (time.time() - start_time) / 1000
        
        avg_loss = total_loss / len(data_loader)
        print(f'Средний loss: {avg_loss}')
        print(f'Время обучения эпохи: {elapsed_time_sec}sec')
    
    def test(self, data_loader:DataLoader):
        self._model.to(device)
        self._model.eval() # переводим модель в режим использования

        total_loss = 0
        total_mcc = 0

        for batch in tqdm(data_loader):
            input_ids = batch[0].to(device)
            attention_mask = batch[1].to(device)
            labels = batch[2].to(device)

            with torch.no_grad():
                pred = self._model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                total_loss += pred.loss.item()
                total_mcc += self._calculate_mcc(pred.logits, labels)
                
        dataset_size = len(data_loader)
        avg_loss = total_loss / dataset_size
        avg_mcc = total_mcc / dataset_size
        print(f'Средний loss: {avg_loss}')
        print(f'Средний MCC: {avg_mcc}')

    def _calculate_mcc(self, logits_pred, y_true):
        pred_flat = np.argmax(logits_pred, axis=1).flatten()
        y_true_flat = y_true.flatten()
        return matthews_corrcoef(y_true_flat, pred_flat)

Загрузим модель.

In [21]:
_bert_model = BertForSequenceClassification.from_pretrained('ai-forever/ruBert-base',
                                                            num_labels = 2,
                                                            output_attentions = False,
                                                            output_hidden_states = False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Токенизируем корпус и обучим модель.

In [35]:
def prepare_dataset(X, y):
    input_ids, attention_masks = RuBertTokenizer().transform(X)
    tensor_y = torch.tensor(y)
    tensor_dataset = TensorDataset(input_ids, attention_masks, tensor_y)
    return DataLoader(dataset=tensor_dataset, batch_size=64, shuffle=True)

_train_data_loader = prepare_dataset(_X_train, _y_train)
_val_data_loader = prepare_dataset(_X_val, _y_val)
_test_data_loader = prepare_dataset(_X_test, _y_test)

_optimizer = AdamW(_bert_model.parameters())
_trainer = Trainer(_bert_model, _optimizer, num_of_epochs=1)

In [37]:
_trainer.train(_train_data_loader)

Эпоха 0...


100%|██████████| 12/12 [04:03<00:00, 20.32s/it]

Средняя ошибка: 0.8132301680743694
Время обучения: 243 days, 21:36:05.837860


In [38]:
_trainer.test(_test_data_loader)

100%|██████████| 16/16 [01:46<00:00,  6.67s/it]

Средняя ошибка: 0.6542324349284172
Средний MCC: 0.0


## Few-/zero-shot с GPT3

## RuT5

### Обучение

### Тестирование